In [1]:
import pandas as pd
import matplotlib.pyplot as mplt
import plotly.express as plt
import numpy as np
import datetime
from bokeh.plotting import figure, show, curdoc
from bokeh.layouts import column, row
from bokeh.models import Select
from bokeh.models.tools import HoverTool
from bokeh.plotting import figure, show, output_file
from bokeh.transform import factor_cmap, factor_mark
from bokeh.themes import built_in_themes
from bokeh.io import curdoc


df = pd.read_parquet("option_chain_data.parquet")

In [2]:
df2 = df[['strike','lastPrice','bid','ask','change','percentChange','volume','openInterest','impliedVolatility']]

summarydf = pd.DataFrame()
for column in df2:
    summarydf[str(column)] = df.agg(
    {
        column: ['min', 'median', 'max', 'mean', 'std']
    }).round(2)
    

summarydf

,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility
min,700.00,0.05,0.00,0.00,-169.05,-42.61,1.00,0.00,0.00
median,1880.00,176.00,189.00,199.70,0.00,0.00,2.00,23.00,1.13
max,4400.00,2277.80,2246.30,2265.00,476.29,73.53,190.00,3710.00,17.75
mean,2071.19,503.59,537.04,548.43,-3.85,0.33,6.99,89.51,2.32
std,873.72,636.11,675.89,679.25,42.49,15.24,18.13,247.45,3.01


In [3]:
df2 = df[['strike','lastPrice','bid','ask','change','percentChange','volume','openInterest','impliedVolatility']]

summarydf = pd.DataFrame()
for column in df2:
    
    temptbl = df.groupby('type').agg(
    {
        column: ['min', 'median', 'max', 'mean', 'std']
    }).swapaxes("index", "columns")
    
    summarydf = pd.concat([summarydf, temptbl], axis = 0)
    

summarydf

type                             Call          Put
strike            min      700.000000   700.000000
                  median  1860.000000  1910.000000
                  max     4400.000000  4300.000000
                  mean    2078.163265  2064.440789
                  std      903.092685   847.290881
lastPrice         min        2.000000     0.050000
                  median   773.850000    17.475000
                  max     2277.800000  1414.320000
                  mean     888.676531   131.174211
                  std      685.573506   245.205132
bid               min        0.300000     0.000000
                  median  1005.800000    16.250000
                  max     2246.300000  1539.000000
                  mean     947.442857   140.145066
                  std      727.990620   258.034716
ask               min        4.300000     0.000000
                  median  1021.000000    23.300000
                  max     2265.000000  1554.800000
                  mean     961.385714   149.064803
                  std      730.328374   261.246731
change            min     -169.050050    -2.200000
                  median     0.000000     0.000000
                  max      476.289920   106.570010
                  mean     -14.929183     6.867105
                  std       55.778955    17.888577
percentChange     min      -42.608982   -16.058393
                  median     0.000000     0.000000
                  max       53.185253    73.529410
                  mean      -6.329114     6.765200
                  std       12.474982    14.937641
volume            min        1.000000     1.000000
                  median     2.000000     2.000000
                  max       72.000000   190.000000
                  mean       5.299320     8.646667
                  std        9.404469    23.683069
openInterest      min        0.000000     0.000000
                  median    15.000000    43.000000
                  max      659.000000  3710.000000
                  mean      72.197279   106.243421
                  std      128.024433   323.140426
impliedVolatility min        0.000000     0.000010
                  median     3.449418     0.493268
                  max       17.746342     1.536379
                  mean       4.218844     0.481486
                  std        3.328963     0.482974

In [7]:
df2 = df[['strike','lastPrice','bid','ask','change','percentChange','volume','openInterest','impliedVolatility', 'type']]
OldRange = (max(df.volume) - min(df.volume))
oldmin = min(df.volume)
NewRange = 50
df['size'] = df['volume'].transform(lambda x:   (((x - x.min()) * NewRange ) / OldRange) + 5 )


#(((df2.volume - oldmin) * NewRange ) / OldRange) + 5
TYPES = df.type.unique()
MARKERS = ['circle', 'triangle']

#contractSymbol lastTradeDate change percentChange
hover = HoverTool(tooltips=[("Contract Symbol", "@contractSymbol")
                           ,("Last Trade Date", "@lastTradeDate")
                           ,("Volume (corresponds to size)", "@volume")
                           ,("Bid", "@bid")
                           ,("Strike", "@strike")
                           ,("Ask", "@ask")
                           ,("Change", "@change")
                           ,("Percent Change", "@percentChange")
                           ])

curdoc().theme = 'dark_minimal'

p = figure(title="Options: bid vs strike", x_axis_label='Bid', y_axis_label='Strike'
         , tools = [hover])

p.scatter('bid', 'strike', source=df
         , marker=factor_mark('type', MARKERS, TYPES)
         , color=factor_cmap('type', 'Category10_3', TYPES)
         , size='size'
         , legend_field = 'type'
         )

p.toolbar.logo=None
# show the results
show(p)